In [1]:
import os
import random
import pickle
import torch
import torchvision
import pandas as pd
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True

In [4]:
import torchvision.transforms as transforms
from torch import nn

In [5]:
class EmbeddingNet(nn.Module):
    """EmbeddingNet using ResNet-101."""

    def __init__(self, resnet):
        """Initialize EmbeddingNet model."""
        super(EmbeddingNet, self).__init__()

        # Everything except the last linear layer
        self.features = nn.Sequential(*list(resnet.children())[:-2])
        num_ftrs = resnet.dense.in_features
        self.dense = nn.Linear(num_ftrs, 4096)

    def forward(self, x):
        """Forward pass of EmbeddingNet."""
        out = self.features(x)
#         out = out.view(out.size(0), -1)
        out = self.dense(out)

        return out


class TripletNet(torch.nn.Module):
    """Triplet Network."""

    def __init__(self, embeddingnet):
        """Triplet Network Builder."""
        super(TripletNet, self).__init__()
        self.embeddingnet = embeddingnet

    def forward(self, a, p, n):
        """Forward pass."""
        # anchor
        embedded_a = self.embeddingnet(a)

        # positive examples
        embedded_p = self.embeddingnet(p)

        # negative examples
        embedded_n = self.embeddingnet(n)

        return embedded_a, embedded_p, embedded_n
    

class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

In [27]:
model = torch.load('../data/pickles/models/23.12.0.20.pth')

In [28]:
embd_net = model.embeddingnet

In [30]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [31]:
# Normalize training set together with augmentation
one_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
#     transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.0069], [0.0033])
])

train_dir = '../data/spectrograms/train/train/'
val_dir = '../data/spectrograms/train/val/'
test_dir = '../data/spectrograms/test/'

batch_size = 16
test_dataset = ImageFolderWithPaths(test_dir, one_transform)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=8
)

train_dataset = ImageFolderWithPaths(train_dir, one_transform)
val_dataset = ImageFolderWithPaths(val_dir, one_transform)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=8
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=8
)

# class_names = train_dataset.classes


# Loading Tiny ImageNet dataset
# trplt_path = '../pickles_for_git/triplets.p'
# batch_szie = 32
# trainset = TripletDataset(triplet_path=trplt_path, transform=one_transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_szie, num_workers=8)

# testset = TripletImageLoader(
#     base_path=root, triplets_filename="", transform=transform_test, train=False)
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=batch_size_test, num_workers=32)

In [32]:
len(test_dataloader), len(test_dataset)

(7, 97)

In [33]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs, x, path in test_loader:
            inputs = inputs.to(device)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = torch.nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [34]:
embd_net = embd_net.to(device)

In [35]:
with torch.no_grad():
    logits = []

    for inputs, x, path in test_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        break

ModuleAttributeError: 'EmbeddingNet' object has no attribute 'dense'

In [14]:
logits[0][0].shape

torch.Size([4096])

In [15]:
with torch.no_grad():
    logits = []
    path_list = list()
    for inputs, x, path in test_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [16]:
with torch.no_grad():
#     logits = []
#     path_list = list()
    for inputs, x, path in val_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [17]:
with torch.no_grad():
#     logits = []
#     path_list = list()
    for inputs, x, path in train_dataloader:
        inputs = inputs.to(device)
        embd_net.eval()
        outputs = embd_net(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [18]:
path_list[0][0]

'../data/spectrograms/test/unknown/000190.png'

In [19]:
logits[0][0].shape[0]

4096

In [20]:
emb_list = list()
for i in logits:
    for tens in i:
        emb_list.append(tens.numpy())

In [21]:
embeddings = pd.DataFrame(emb_list)

In [22]:
track_list = list()
for i in path_list:
    for path in i:
        name = path.split('/')[-1]
        ids = name.split('.')[0]
        track_list.append(ids)

In [23]:
embeddings['id'] = track_list

In [24]:
embeddings = embeddings[['id'] + list(np.arange(logits[0][0].shape[0]))]

In [25]:
embeddings

,id,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,000190,-1.276893,0.408302,0.291343,1.445127,0.996394,1.410408,-1.059028,-0.770857,-1.551588,...,0.829957,-1.234529,1.042391,0.312923,1.134506,-0.318664,-0.097139,0.573347,-0.352778,-0.140766
1,000194,-1.201204,0.363852,0.306437,1.436126,0.790721,1.532884,-1.076252,-0.828838,-1.447008,...,0.813164,-1.331910,1.028743,0.469838,1.227907,-0.257681,0.099148,0.683309,-0.243152,-0.111059
2,000667,-0.731311,-0.059756,0.043560,0.905223,0.973806,1.078042,-1.005939,-0.899546,-1.736002,...,0.443425,-0.823968,0.926827,0.747362,0.813988,-0.668154,-0.222066,0.720436,-0.123378,0.251941
3,001040,-0.930320,-0.344406,0.097284,0.841191,0.997342,1.034332,-1.212615,-0.921806,-1.487170,...,0.256667,-1.023019,0.620918,0.537424,0.959706,-0.558905,-0.188720,0.478831,0.025207,0.287220
4,001686,-0.840700,0.000242,0.204627,1.189205,0.889482,1.353601,-1.096324,-0.894396,-1.519699,...,0.338232,-0.943736,1.311060,0.589391,1.155286,-0.379741,0.113926,0.895250,-0.688248,-0.257464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7989,149416,-1.198173,-0.121649,0.101514,1.089058,0.838044,1.161070,-1.322232,-0.867880,-1.447068,...,0.334303,-1.067471,0.522970,0.386904,0.821854,-0.622950,-0.331727,0.344130,-0.144897,0.354650
7990,149417,-1.406448,-0.155011,0.223302,1.244829,0.684316,1.172671,-1.152496,-0.825584,-1.393516,...,0.463438,-1.044278,0.755577,-0.023398,1.021887,-0.581454,-0.363187,0.413588,-0.117126,0.255743
7991,149452,-1.108290,-0.116360,0.258910,0.874292,1.191722,1.027386,-1.324970,-0.913047,-1.032652,...,0.284604,-0.914654,1.015525,0.559520,1.123050,-0.592272,-0.406806,0.344648,-0.185501,0.395425
7992,149488,-1.120759,0.297402,0.120046,0.615894,0.744380,1.431480,-1.428654,-0.602216,-1.262241,...,0.527568,-0.822180,0.837380,0.738472,1.187538,-0.778346,-0.043030,0.591897,-0.388566,0.131399


In [26]:
embeddings.to_csv('../data/csv/triplet_embeddings.csv', index=False)